In [1]:
from nipype import Node, Workflow, IdentityInterface, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
from nipype.algorithms.rapidart import ArtifactDetect as ART
import numpy as np
import os

In [2]:
data = "/media/paradeisios/UUI/fmri/flanker/raw"
output = "/media/paradeisios/UUI/fmri/flanker/output/preprocessing"

num_of_subjects = 26
num_of_runs = 2

subject_list = ["{:02d}".format(i) for i in range(1,num_of_subjects+1)]

mni_template = "/home/paradeisios/fsl/data/standard/MNI152_T1_2mm.nii.gz"
mni_brain_template = "/home/paradeisios/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz"

TR = 2

In [3]:
infosource = Node(IdentityInterface(fields=["subject_list"]), name="infosource")
infosource.iterables = [('subject_list', subject_list)]

In [4]:
templates = {"anat": "sub-{subject_list}/anat/sub-{subject_list}_T1w.nii.gz",
             "func": "sub-{subject_list}/func/sub-{subject_list}_task-flanker_run-*_bold.nii.gz",
             "conditions": "sub-{subject_list}/func/sub-{subject_list}_task-flanker_run-*_events.tsv"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data


substitutions = [("_mean_img","mean_img"),
                 ("_subject_list",""),
                 ("_flirt_func","flirt_func"),
                 ("_func_warp","func_warp")]

substitutions += [("_{:02d}".format(i), "{:02d}".format(i)) for i in range(1,num_of_subjects+1)]
substitutions += [("_art{:d}".format(i), "art_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("mean_img{:d}".format(i), "mean_img_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_realign{:d}".format(i), "realign_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("flirt_func{:d}".format(i), "flirt_func_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("func_warp{:d}".format(i), "func_warp_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_smooth{:d}".format(i), "smooth_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_vol_removal{:d}".format(i), "vol_removal_{:d}".format(i+1)) for i in range(num_of_runs)]

substitutions += [("seg_0", "GM")]
substitutions += [("seg_1", "WM")]
substitutions += [("seg_2", "CSF")]

datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

In [5]:
# structural workflow 

bet = Node(fsl.BET(),name="bet")
bet.inputs.frac = 0.3 #value controlling how much of the skull to be removed - increasing might remove bits of brain
bet.inputs.vertical_gradient = 0.2 #value to linearly improve estimations at the bottom of the brain -increasing might understimate top
bet.inputs.mask = True #creates a binary mask of the brain


segment = Node(fsl.FAST(),name="segment")
"""You are interested mainly in the restored version of the structural and
    pve0 = grey matter
    pve1 = white matter
    pve2 = CSF """
segment.inputs.img_type = 1 # indicate its a T1 image
segment.inputs.number_classes = 3 # seperate structural into Grey Matter, White Matter and CSF
segment.inputs.output_biascorrected = True #return structural image bias field corrected
segment.inputs.output_biasfield = True #return bias field
segment.inputs.segments=True #create a binary mask for each tissue type
segment.inputs.no_pve=False #if you dont want partial volume estimation, set to true

flirt_struct = Node(fsl.FLIRT(),name="flirt_struct")
"""This step performs linear registration of the structural volume to the standard space"""
flirt_struct.inputs.reference = mni_brain_template
flirt_struct.inputs.dof = 12 #use 12 dof in structural flirt to prepare image for FNIRT nonlinear normalization to mni space
flirt_struct.inputs.out_matrix_file = "h2s_affine.mat" #keep this to facilite struct and func normalization to mni space using FNIRT
flirt_struct.inputs.interp = "spline" # change to trilinear if you are mostly interested in non-subcortical normalization. Trilinear is faster, but might have some rotational issues
flirt_struct.inputs.cost = "mutualinfo"


fnirt_struct = Node(fsl.FNIRT(),name="fnirt_struct")
"""This step performs non-linear registration of the structural volume to the standard space"""
fnirt_struct.inputs.ref_file = mni_template
fnirt_struct.inputs.config_file = "T1_2_MNI152_2mm"
fnirt_struct.inputs.fieldcoeff_file = True  #keep


In [6]:
# functional workflow

img_to_float = MapNode(fsl.ImageMaths(),name="img_to_float",iterfield=['in_file'])
"""This converts the 4d in float representation to faciliate some further computations"""
img_to_float.inputs.out_data_type='float'
img_to_float.inputs.op_string=''


vol_removal = MapNode(fsl.ExtractROI(), name="vol_removal",iterfield=['in_file'])
"""This step removes the 5 first volumes to avoid T1 saturation"""
vol_removal.inputs.t_min = 0 #number of scans to exclude from functional image
vol_removal.inputs.t_size = -1 # keep all the rest until the end


realign = MapNode(fsl.MCFLIRT(),name = "realign",iterfield=['in_file'])
"""This step realigns data to the first image"""
realign.inputs.save_mats = True #save realignment  parameters
realign.inputs.save_plots = True #save parameter plots
realign.inputs.dof = 6 #use this for rigid body correction


mean_img = MapNode(fsl.maths.MeanImage(),name= "mean_img",iterfield=['in_file'])
mean_img.inputs.dimension = "T" # find mean image across the time dimension

func_bet = MapNode(fsl.BET(),name= "func_bet",iterfield=['in_file'])
func_bet.inputs.frac = 0.3 #value controlling how much of the skull to be removed - increasing might remove bits of brain
func_bet.inputs.mask = True #creates a binary mask of the brain

art = MapNode(ART(),name="art",iterfield=["mask_file","realigned_files","realignment_parameters"])
art.inputs.mask_type="file"
art.inputs.norm_threshold=1
art.inputs.zintensity_threshold = 3
art.inputs.parameter_source = "FSL"
art.inputs.use_norm=True
art.inputs.use_differences = [True, False]



flirt_func = MapNode(fsl.FLIRT(),name = "flirt_func",iterfield=['in_file'])
"""This step performs linear registration of the functional volume to the structural volume. It is mainly used to
extract the transformation matrix to later register to standard space"""
flirt_func.inputs.dof = 6
flirt_func.inputs.out_matrix_file = "f2h_affine.mat"
flirt_func.inputs.interp = "spline"
flirt_func.inputs.cost = "mutualinfo"


func_warp = MapNode(fsl.ApplyWarp(),name = "func_warp",
                    iterfield=['in_file',"premat"])
"""This step performs non linear registration to standard space using the affine matrix extracted
in the flirt above and the fnirt non linear warp in the structural pipeline """
func_warp.inputs.ref_file = mni_template


smooth = MapNode(fsl.IsotropicSmooth(),name="smooth",iterfield=['in_file'])
"""This step performs smoothing"""
smooth.inputs.fwhm = 6 #change the smoothing kernel

In [7]:
preprocessing = Workflow(name="preprocessing",base_dir=output)

preprocessing.connect(infosource,"subject_list",selectfiles, "subject_list")
preprocessing.connect(infosource,"subject_list",datasink, "container")                  
preprocessing.connect(selectfiles,"anat",bet, "in_file")  
preprocessing.connect(selectfiles,"anat",fnirt_struct, "in_file") 
                 
preprocessing.connect(bet,"out_file",segment,"in_files")
preprocessing.connect(segment,"restored_image",flirt_struct,"in_file")
preprocessing.connect(flirt_struct,"out_matrix_file",fnirt_struct,"affine_file")

preprocessing.connect(selectfiles,"func",img_to_float, "in_file")   
preprocessing.connect(img_to_float,"out_file",vol_removal,"in_file")
preprocessing.connect(vol_removal, "roi_file", realign, "in_file")
preprocessing.connect(realign, "out_file", mean_img, "in_file")

preprocessing.connect(mean_img, "out_file", func_bet, "in_file")
preprocessing.connect(func_bet, "mask_file", art, "mask_file")
preprocessing.connect(realign, "out_file", art, "realigned_files")
preprocessing.connect(realign, "par_file", art, "realignment_parameters")

preprocessing.connect(mean_img,"out_file",flirt_func,"in_file")
preprocessing.connect(realign,"out_file",func_warp,"in_file")
preprocessing.connect(flirt_func,"out_matrix_file",func_warp,"premat")
preprocessing.connect(segment,"restored_image",flirt_func,"reference")
preprocessing.connect(fnirt_struct,"fieldcoeff_file",func_warp,"field_file")
preprocessing.connect(func_warp,"out_file",smooth,"in_file")

In [8]:
preprocessing.write_graph(graph2use='flat',format='png', simple_form=True,
                          dotfilename= os.path.join(output,"preprocessing_workflow.dot"))

210813-00:26:49,875 nipype.workflow INFO:
	 Generated workflow graph: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing_workflow.png (graph2use=flat, simple_form=True).


'/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing_workflow.png'

In [9]:
preprocessing.connect(selectfiles,"conditions",datasink,"conditions")
preprocessing.connect(bet,"out_file",datasink,"bet.@bet_brain")
preprocessing.connect(bet,"mask_file",datasink,"bet.@bet_mask")
preprocessing.connect(segment,"restored_image",datasink,"segment.@restored_image")
preprocessing.connect(segment,"tissue_class_files",datasink,"segment.@tissue_class_files")
preprocessing.connect(segment,"bias_field",datasink,"segment.@bias_field")
preprocessing.connect(segment,"tissue_class_map",datasink,"segment.@tissue_class_map")
preprocessing.connect(flirt_struct,"out_file",datasink,"flirt_struct.@structural_flirt")
preprocessing.connect(flirt_struct,"out_matrix_file",datasink,"flirt_struct.@affine_parameters")
preprocessing.connect(fnirt_struct,"warped_file",datasink,"flirt_struct.@structural_fnirt")
preprocessing.connect(fnirt_struct,"field_file",datasink,"flirt_struct.@fieldcoeff_file")
preprocessing.connect(vol_removal,"roi_file",datasink,"vol_removal.@files")
preprocessing.connect(realign,"out_file",datasink,"realign.@realigned_files")
preprocessing.connect(realign,"par_file",datasink,"realign.@realig_parameters")
preprocessing.connect(art,"outlier_files",datasink,"art.@art_outliers")
preprocessing.connect(art,"plot_files",datasink,"art.@art_plot")
preprocessing.connect(mean_img,"out_file",datasink,"mean_img.@mean_img")
preprocessing.connect(flirt_func,"out_matrix_file",datasink,"flirt_func.@affine_parameters")
preprocessing.connect(flirt_func,"out_file",datasink,"flirt_func.@functional_flirt")
preprocessing.connect(func_warp,"out_file",datasink,"func_warp.@functional_fnirt")
preprocessing.connect(smooth,"out_file",datasink,"smooth.@smoothed_files")

In [10]:
preprocessing.run(plugin='MultiProc', plugin_args={'n_procs' : 4})

210813-00:26:50,76 nipype.workflow INFO:
	 Workflow preprocessing settings: ['check', 'execution', 'logging', 'monitoring']
210813-00:27:41,363 nipype.workflow INFO:
	 Running in parallel.
210813-00:27:41,399 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 26 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 4/4.
210813-00:27:41,633 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.selectfiles" in "/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_26/selectfiles".
210813-00:27:41,703 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.selectfiles" in "/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_25/selectfiles".
210813-00:27:41,741 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.selectfiles" in "/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_24/selectfiles".
210813-00:27:41,798 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.sel

210813-00:39:25,282 nipype.workflow INFO:
	 [Node] Finished "preprocessing.segment".
210813-00:39:26,118 nipype.workflow INFO:
	 [Job 23] Completed (preprocessing.segment).
210813-00:39:26,173 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 31 jobs ready. Free memory (GB): 6.14/6.74, Free processors: 1/4.
                     Currently running:
                       * preprocessing.segment
                       * preprocessing.segment
                       * preprocessing.segment
210813-00:39:26,371 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.flirt_struct" in "/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_25/flirt_struct".
210813-00:39:26,449 nipype.workflow INFO:
	 [Node] Running "flirt_struct" ("nipype.interfaces.fsl.preprocess.FLIRT"), a CommandLine Interface with command:
flirt -in /media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_25/segment/sub-25_T1w_brain_restore.nii.gz -ref /home/

210813-00:51:30,844 nipype.workflow INFO:
	 [Job 40] Completed (preprocessing.flirt_struct).
210813-00:51:30,854 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 31 jobs ready. Free memory (GB): 6.14/6.74, Free processors: 1/4.
                     Currently running:
                       * preprocessing.fnirt_struct
                       * preprocessing.flirt_struct
                       * preprocessing.flirt_struct
210813-00:51:30,924 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.fnirt_struct" in "/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_24/fnirt_struct".
210813-00:51:31,4 nipype.workflow INFO:
	 [Node] Running "fnirt_struct" ("nipype.interfaces.fsl.preprocess.FNIRT"), a CommandLine Interface with command:
fnirt --aff=/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_24/flirt_struct/h2s_affine.mat --config=T1_2_MNI152_2mm --cout=/media/paradeisios/UUI/fmri/flanker/output/preprocessing

210813-01:09:39,680 nipype.workflow INFO:
	 [Node] Finished "preprocessing.bet".
210813-01:09:39,962 nipype.workflow INFO:
	 [Job 67] Completed (preprocessing.bet).
210813-01:09:39,981 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 32 jobs ready. Free memory (GB): 6.14/6.74, Free processors: 1/4.
                     Currently running:
                       * preprocessing.fnirt_struct
                       * preprocessing.fnirt_struct
                       * preprocessing.fnirt_struct
210813-01:09:40,60 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.segment" in "/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_22/segment".
210813-01:09:40,86 nipype.workflow INFO:
	 [Node] Running "segment" ("nipype.interfaces.fsl.preprocess.FAST"), a CommandLine Interface with command:
fast -t 1 -n 3 -B -b -g -S 1 /media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_22/segment/sub-22_T1w_brain.nii.gz
210813-01:0

210813-01:13:22,437 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.selectfiles" in "/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_19/selectfiles".
210813-01:13:22,469 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210813-01:13:22,826 nipype.workflow INFO:
	 [Node] Finished "preprocessing.selectfiles".
210813-01:13:24,196 nipype.workflow INFO:
	 [Job 105] Completed (preprocessing.selectfiles).
210813-01:13:24,263 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 34 jobs ready. Free memory (GB): 6.14/6.74, Free processors: 1/4.
                     Currently running:
                       * preprocessing.segment
                       * preprocessing.segment
                       * preprocessing.segment
210813-01:13:24,488 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.bet" in "/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_19/bet".
210813-0

...

210813-07:27:17,177 nipype.workflow INFO:
	 [Node] Finished "_smooth0".
210813-07:27:17,867 nipype.workflow INFO:
	 [Job 854] Completed (_smooth0).
210813-07:27:17,875 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 1 jobs ready. Free memory (GB): 6.14/6.74, Free processors: 1/4.
                     Currently running:
                       * _smooth0
                       * _smooth1
                       * _smooth1
210813-07:27:17,987 nipype.workflow INFO:
	 [Node] Setting-up "_smooth1" in "/media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_01/smooth/mapflow/_smooth1".
210813-07:27:18,12 nipype.workflow INFO:
	 [Node] Running "_smooth1" ("nipype.interfaces.fsl.maths.IsotropicSmooth"), a CommandLine Interface with command:
fslmaths /media/paradeisios/UUI/fmri/flanker/output/preprocessing/preprocessing/_subject_list_01/func_warp/mapflow/_func_warp1/sub-01_task-flanker_run-2_bold_maths_roi_mcf_warp.nii.gz -s 2.54797 /media/paradeisios/UUI/fmr

210813-07:27:37,420 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/03/mean_img/_subject_list_03/_mean_img0/sub-03_task-flanker_run-1_bold_maths_roi_mcf_mean.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/03/mean_img/03/mean_img_run_1/sub-03_task-flanker_run-1_bold_maths_roi_mcf_mean.nii.gz
210813-07:27:37,470 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/03/mean_img/_subject_list_03/_mean_img1/sub-03_task-flanker_run-2_bold_maths_roi_mcf_mean.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/03/mean_img/03/mean_img_run_2/sub-03_task-flanker_run-2_bold_maths_roi_mcf_mean.nii.gz
210813-07:27:37,517 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/03/art/_subject_list_03/_art0/art.sub-03_task-flanker_run-1_bold_maths_roi_mcf_outliers.txt -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/03/art/03/art_run_1/art.sub-03_task-fla

210813-07:28:32,718 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/flirt_struct/_subject_list_02/sub-02_T1w_warped.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/flirt_struct/02/sub-02_T1w_warped.nii.gz
210813-07:28:32,722 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/segment/_subject_list_02/sub-02_T1w_brain_restore.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/segment/02/sub-02_T1w_brain_restore.nii.gz
210813-07:28:32,731 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/segment/_subject_list_02/sub-02_T1w_brain_seg_0.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/segment/02/sub-02_T1w_brain_GM.nii.gz
210813-07:28:32,735 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/segment/_subject_list_02/sub-02_T1w_brain_seg_1.nii.gz -> /media/paradeisios/UUI/f

210813-07:28:32,973 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/smooth/_subject_list_02/_smooth0/sub-02_task-flanker_run-1_bold_maths_roi_mcf_warp_smooth.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/smooth/02/smooth_run_1/sub-02_task-flanker_run-1_bold_maths_roi_mcf_warp_smooth.nii.gz
210813-07:28:32,981 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/smooth/_subject_list_02/_smooth1/sub-02_task-flanker_run-2_bold_maths_roi_mcf_warp_smooth.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/02/smooth/02/smooth_run_2/sub-02_task-flanker_run-2_bold_maths_roi_mcf_warp_smooth.nii.gz
210813-07:28:33,28 nipype.workflow INFO:
	 [Node] Finished "preprocessing.datasink".
210813-07:28:33,937 nipype.workflow INFO:
	 [Job 374] Completed (preprocessing.datasink).
210813-07:28:33,947 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 6.

210813-07:29:08,816 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/01/mean_img/_subject_list_01/_mean_img1/sub-01_task-flanker_run-2_bold_maths_roi_mcf_mean.nii.gz -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/01/mean_img/01/mean_img_run_2/sub-01_task-flanker_run-2_bold_maths_roi_mcf_mean.nii.gz
210813-07:29:08,824 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/01/art/_subject_list_01/_art0/art.sub-01_task-flanker_run-1_bold_maths_roi_mcf_outliers.txt -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/01/art/01/art_run_1/art.sub-01_task-flanker_run-1_bold_maths_roi_mcf_outliers.txt
210813-07:29:08,829 nipype.interface INFO:
	 sub: /media/paradeisios/UUI/fmri/flanker/output/preprocessing/01/art/_subject_list_01/_art1/art.sub-01_task-flanker_run-2_bold_maths_roi_mcf_outliers.txt -> /media/paradeisios/UUI/fmri/flanker/output/preprocessing/01/art/01/art_run_2/art.sub-01_task-flanker_run-2